# sveltejs/sveltekit

> 

In [ ]:
from lovely_docs.settings import settings, GitSource
from lovely_docs.git import clone_repo
from lovely_docs.docs import (
    build_markdown_doc_tree,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents
)
import logging

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#### Clone the repo

In [ ]:
source = GitSource(
    name="sveltejs/sveltekit",
    doc_dir="apps/svelte.dev/content/docs/kit",
    repo="https://github.com/sveltejs/svelte.dev",
    commit="main",
    ecosystems=["svelte"])

commit, clone_dir = clone_repo(source)
source.commit = commit # Replace the literal commit (master) with the hash

#### Read the markdown files

In [ ]:
tree = build_markdown_doc_tree(clone_dir/source.doc_dir)
# tree.model_dump()

#### Process the tree

In [ ]:
# tree.subdirs = tree.subdirs[:2] # For testing process just the first 2 secstion (intro and runes)

In [ ]:
processed_tree = await process_tree_depth_first(settings, tree, source.name)

+0.889s DEBUG lovely_docs/docs.py:153 llm_process_page Processing 10-getting-started/10-introduction.md
+0.250s DEBUG lovely_docs/docs.py:153 llm_process_page Processing 10-getting-started/20-creating-a-project.md
+0.016s DEBUG lovely_docs/docs.py:153 llm_process_page Processing 10-getting-started/25-project-types.md
+0.015s DEBUG lovely_docs/docs.py:153 llm_process_page Processing 10-getting-started/30-project-structure.md
+0.017s DEBUG lovely_docs/docs.py:153 llm_process_page Processing 10-getting-started/40-web-standards.md
+0.016s DEBUG lovely_docs/docs.py:153 llm_process_page Processing 10-getting-started/index.md
+0.017s DEBUG lovely_docs/docs.py:153 llm_process_page Processing 20-core-concepts/10-routing.md
+0.016s DEBUG lovely_docs/docs.py:153 llm_process_page Processing 20-core-concepts/20-load.md
+0.018s DEBUG lovely_docs/docs.py:153 llm_process_page Processing 20-core-concepts/30-form-actions.md
+0.018s DEBUG lovely_docs/docs.py:153 llm_process_page Processing 20-core-concep

#### Calculate cost

In [ ]:
usage =calculate_total_usage(processed_tree)
print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")

cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")


Total Usage:
  Input tokens: 333,462
  Output tokens: 100,146
  Total tokens: 433,608
Cost:
  Total: $0.83
  Input: $0.33
  Output: $0.50


#### save the results

In [ ]:
save_processed_documents(source, settings.output_dir / source.name.replace("/", "_"), processed_tree)